### Part 2: change loss function from $loss = \frac{1}{n}\sum{(y_i - \hat(y_i))^2}$ to $loss = \frac{1}{n}\sum{|y_i - \hat{y_i}|}$, and using your mathmatical knowledge to get the right partial formual. Implemen the gradient descent code.

In [11]:
from sklearn.datasets import load_boston
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
%matplotlib inline

In [12]:
data = load_boston()
X, y = data['data'], data['target']

In [36]:
def get_prices(k, b, x):
    return np.dot(x, k) + b

def get_loss(y, y_hat): # to evaluate the performance 
    return sum(abs(y_i - y_hat_i) for y_i, y_hat_i in zip(list(y), list(y_hat))) / len(list(y))

def partial_k(x, y, y_hat):
    n = len(y)
    gradient = 0
    for x, y, y_hat in zip(list(x), list(y), list(y_hat)):
        if y > y_hat:
            gradient += x/n
        elif y < y_hat:
            gradient -= x/n
    return -gradient

def partial_b(x, y, y_hat):
    n = len(y)
    gradient = 0
    for x, y, y_hat in zip(list(x), list(y), list(y_hat)):
        if y > y_hat:
            gradient += 1/n
        elif y < y_hat:
            gradient -= 1/n
    
    return -gradient

In [49]:


k = best_supervised_k = [random.random() * 200 - 100 for i in range(13)]
b = best_supervised_b = random.random() * 200 - 100

step = 1e-3
epoch = 100000
lowest_loss = 10000000000000

for i in range(epoch):
    y_hat = get_prices(k, b, X)
    k -= partial_k(X, y, y_hat) * step
    b -= partial_b(X, y, y_hat) * step
    loss = get_loss(y, y_hat)
    if i % 5000 == 0:
        print(loss)
    if loss < lowest_loss:
        lowest_loss = loss
        best_directed_k = k
        best_directed_b = b
        if i % (epoch/10) == 0:
            print('new lowest loss:{0} at {1}'.format(lowest_loss, i))
print(lowest_loss)

26803.420318210516
new lowest loss:26803.420318210516 at 0
812.22204394917
282.9759698628897
new lowest loss:282.9759698628897 at 10000
245.32783928133142
211.08786384273895
new lowest loss:211.08786384273895 at 20000
178.53273309418006
147.3583591352219
118.4090160298157
116.81656475877394
142.6233388211896
141.5771248511659
143.80762115204183
145.840274714179
152.04021415056206
153.59352158270497
158.03244867031484
159.56931654498015
155.92981670468504
156.62062015390956
157.06601047986516
112.56221977040954


### Part 3: Finish the Solution Parse Part of Edit-Distance

In [78]:
from functools import lru_cache

In [255]:
class edit_distance():
    '''
    compute是老师例子里面的“edit_distance”
    '''
    def __init__(self, string1, string2, solution = {}):
        self.solution = {}
        self.parsed_solution = []
        self.string1 = string1
        self.string2 = string2
        self.string_used = False
    
    @lru_cache(maxsize=2**10)
    def compute(self, string1 = '', string2 = ''):
        if self.string_used == False:
            string1 = self.string1
            string2 = self.string2
            self.string_used = True
        if len(string1) == 0: 
            self.solution[(string1, string2)] = 'ADD {}'.format(string2)
            self.string_used = False
            return len(string2)
        if len(string2) == 0: 
            self.solution[(string1, string2)] = 'DEL {}'.format(string1)
            self.string_used = False
            return len(string1)

        tail_s1 = string1[-1]
        tail_s2 = string2[-1]

        candidates = [
            (self.compute(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
            (self.compute(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
        ]

        if tail_s1 == tail_s2:
            both_forward = (self.compute(string1[:-1], string2[:-1]) + 0, '')
        else:
            both_forward = (self.compute(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

        candidates.append(both_forward)

        min_distance, operation = min(candidates, key=lambda x: x[0])

        self.solution[(string1, string2)] = operation 

        return min_distance
    
    @lru_cache(maxsize=2**10)
    def parse_solution(self, string1 = '', string2 = ''):
        if self.string_used == False:
            string1 = self.string1
            string2 = self.string2
            self.string_used = True
            
        operation = self.solution[string1, string2]
        if len(string1) == 0: 
            self.string_used = False
            self.parsed_solution.append(operation + ' before ind={}'.format(len(string1)))
            return self.parsed_solution[::-1]
        if len(string2) == 0: 
            self.string_used = False
            self.parsed_solution.append(operation + ' before ind={}'.format(len(string1)))
            return self.parsed_solution[::-1]
        
        if operation == '':
            self.parse_solution(string1[:-1], string2[:-1])
        else:
            operator = operation[:3]
            operand = operation[3:]
            if operator == 'ADD':
                self.parse_solution(string1, string2[:-1])
                self.parsed_solution.append(operation + ' after ind={}'.format(len(string1)-1))
            elif operator == 'DEL':
                self.parse_solution(string1[:-1], string2)
                self.parsed_solution.append(operation + ' at ind={}'.format(len(string1)-1))
            elif operator == 'SUB':
                self.parse_solution(string1[:-1], string2[:-1])
                self.parsed_solution.append(operation + ' at ind={}'.format(len(string1)-1))
        return self.parsed_solution[::-1]

In [256]:
distance = edit_distance('i like','you like the banana')
distance.compute()
distance.parse_solution()

['ADD a after ind=5',
 'ADD n after ind=5',
 'ADD a after ind=5',
 'ADD n after ind=5',
 'ADD a after ind=5',
 'ADD b after ind=5',
 'ADD   after ind=5',
 'ADD e after ind=5',
 'ADD h after ind=5',
 'ADD t after ind=5',
 'ADD   after ind=5',
 'SUB i => u at ind=0',
 'ADD yo before ind=0']

### (Optinal) Finish the k-person-salesman problem: